In [1]:
using CompTime

In [2]:
include("MacroUtils.jl")
using .MacroUtils: cleanup

In [3]:
function dfa_parity(xs)
    ys = ()
    state = :s0
    for x in xs
        if state == :s0
            x == 0 && (ys = (ys..., :even); state = :s0; continue)
            x == 1 && (ys = (ys..., :odd); state = :s1; continue)
            error("Unexpected input: ", x)
        elseif state == :s1
            x == 0 && (ys = (ys..., :odd); state = :s1; continue)
            x == 1 && (ys = (ys..., :even); state = :s0; continue)
            error("Unexpected input: ", x)
        end
    end
    return ys
end

dfa_parity (generic function with 1 method)

In [4]:
dfa_parity((0, 1, 0, 1, 0))

(:even, :odd, :odd, :even, :even)

In [5]:
function dfa_parity_goto(xs)
    ys = ()
    xs_it = iterate(xs)
    @goto s0

    @label s0
    xs_it !== nothing || return ys
    x, xs_state = xs_it
    xs_it = iterate(xs, xs_state)
    x == 0 && (ys = (ys..., :even); @goto s0)
    x == 1 && (ys = (ys..., :odd); @goto s1)
    error("Unexpected input: ", x)

    @label s1
    xs_it !== nothing || return ys
    x, xs_state = xs_it
    xs_it = iterate(xs, xs_state)
    x == 0 && (ys = (ys..., :odd); @goto s1)
    x == 1 && (ys = (ys..., :even); @goto s0)
    error("Unexpected input: ", x)
end

dfa_parity_goto (generic function with 1 method)

In [6]:
dfa_parity_goto((0, 1, 0, 1, 0))

(:even, :odd, :odd, :even, :even)

In [7]:
# @code_lowered dfa_parity_goto((0, 1, 0, 1, 0))

In [8]:
parity_table =
    (:s0, (
        (0, :s0, :even),
        (1, :s1, :odd))),
    (:s1, (
        (0, :s1, :odd),
        (1, :s0, :even)))

((:s0, ((0, :s0, :even), (1, :s1, :odd))), (:s1, ((0, :s1, :odd), (1, :s0, :even))))

In [9]:
function dfa_gen_impl(::Val{tbl}, ::Val{start}, xs) where {tbl,start}
    es = Expr[]

    for (s, tbl_s) in tbl
        push!(es, :(@label $s))
        push!(es, :(xs_it !== nothing || return ys))
        push!(es, :((x, xs_state) = xs_it))
        push!(es, :(xs_it = iterate(xs, xs_state)))
        for (x0, g, w) in tbl_s
            push!(es, :(x == $x0 && (ys = (ys..., $(Expr(:quote, w))); @goto $g)))
        end
        push!(es, :(error("Unexpected input: ", x)))
    end

    return quote
        ys = ()
        xs_it = iterate(xs)
        @goto $start
        $(es...)
    end
end

dfa_gen_impl (generic function with 1 method)

In [10]:
dfa_gen_impl(Val(parity_table), Val(:s0), (0, 1, 0, 1, 0)) |> cleanup

quote
    ys = ()
    xs_it = iterate(xs)
    @goto s0
    @label s0
    xs_it !== nothing || return ys
    (x, xs_state) = xs_it
    xs_it = iterate(xs, xs_state)
    x == 0 && begin
            ys = (ys..., :even)
            @goto s0
        end
    x == 1 && begin
            ys = (ys..., :odd)
            @goto s1
        end
    error("Unexpected input: ", x)
    @label s1
    xs_it !== nothing || return ys
    (x, xs_state) = xs_it
    xs_it = iterate(xs, xs_state)
    x == 0 && begin
            ys = (ys..., :odd)
            @goto s1
        end
    x == 1 && begin
            ys = (ys..., :even)
            @goto s0
        end
    error("Unexpected input: ", x)
end

In [11]:
@generated function dfa_gen(::Val{tbl}, ::Val{start}, xs) where {tbl,start}
    dfa_gen_impl(Val(tbl), Val(start), xs)
end

dfa_gen (generic function with 1 method)

In [12]:
dfa_gen(Val(parity_table), Val(:s0), (0, 1, 0, 1, 0))

(:even, :odd, :odd, :even, :even)

In [13]:
# `@label @ct( )`` and `@goto @ct( )`` doesn't work as expected!
# Besides, how to turn ``:s0`` to `s0`?

#= 
@ct_enable function dfa_gen_ct(@ct(tbl), @ct(start), xs)
    ys = ()
    xs_it = iterate(xs)
    @goto @ct(start)

    @ct_ctrl for (s, tbl_s) in tbl
        @label @ct(s)
        xs_it !== nothing || return ys
        (x, xs_state) = xs_it
        xs_it = iterate(xs, xs_state)
        @ct_ctrl for (x0, g, r) in tbl_s
            x == @ct(x0) && begin
                ys = (ys..., @ct(r))
                @goto @ct(g)
            end
        end
        error("Unexpected input: ", x)
    end
end
 =#

In [14]:
# debug(dfa_gen_ct, Val{parity_table}, Val{:s0}, (0, 1, 0, 1, 0)) |> cleanup

In [15]:
# dfa_gen_ct(Val{parity_table}, Val{:s0}, (0, 1, 0, 1, 0))